# FIRE 

This notebook will try to answer the following questions:

- Given my current savings and no income, how many months will my savings last


## Assumptions

- The annual return on the stock investment (assumption ETF) is an average and is applied monthly. 
The assumption is a long term.
- The bond investment has a fixed return rate that's applied monthly. In reality that should be close to inflation rate.
- The monthly income is constant and is added to cash every month. It's also a simplification as in reality the income is not constant
- The monthly expenses are constant but the inflation rate is applied. In reality, the expenses won't be constant as they change depending on many factors.
- The inflation rate is constant
- When the cash is depleted, the investment is used to cover the expenses. The investment is used in the following order: cash, bond, stock, real estate. 
- Simulation ends when the wealth is depleted and no more assets are available to cover the expenses.


In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [2]:
import datetime
from fire.simulations import FireSimulation, run_simulation, InvestmentProperty
import pandas as pd
import plotly.express as px
from decimal import Decimal


In [3]:


init = FireSimulation(
    stock_investments=100_000,
    bonds_investments=100_000,
    cash=100_000,
    investment_properties=[
        InvestmentProperty(
            mortgage_left=Decimal("292000"),
            mortgage_rate=Decimal("7.66"),
            mortgage_months=202,
            market_value=Decimal("520_000"),
            monthly_income=Decimal("2_100"),
        )
    ],
    stock_return_rate=0.05,
    bonds_return_rate=0.04,
    monthly_expenses=16_000,
    annual_inflation_rate=0.04,
    monthly_income=0,
    annual_property_appreciation_rate=Decimal('0.02'),
    invest_cash_surplus=True,
    invest_cash_threshold=50_000,
    invest_cash_surplus_strategy='80-20',
    date=datetime.datetime.fromisoformat('2024-03-01')
)

# simulate for next X years
years = 15
expected_number_of_months = years * 12
simulation = run_simulation(init, years*12)

In [4]:

df = pd.DataFrame([s.to_dict() for s in simulation])

number_of_records = len(df)
last_record = df.iloc[-1]
df


,stock_investments,bonds_investments,cash,monthly_expenses,monthly_income,date,return_rate_from_investment,investment_properties,return_rate_from_bonds,annual_inflation_rate,annual_property_appreciation_rate,invest_cash_surplus,invest_cash_threshold,invest_cash_surplus_strategy,liquid_wealth,wealth_inc_properties,properties_monthly_mortgage,properties_market_value,properties_monthly_income,properties_net_cash_value
0,100000.000000,100000.000000,100000.000000,16000.000000,0,2024-03-01,0.05,"[{'market_value': 520000, 'monthly_income': 21...",0.04,0.04,0.02,True,50000,80-20,300000.000000,528000.000000,2576.48,520000.00,2100.0,228000.00
1,129254.003333,107542.663333,50000.000000,16053.333333,0,2024-04-01,0.05,"[{'market_value': 520866.67, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,286796.666667,516375.886667,2576.48,520866.67,2100.0,229579.22
2,129792.560000,107901.140000,35993.155556,16106.844444,0,2024-05-01,0.05,"[{'market_value': 521734.78, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,273686.855556,504851.285556,2576.48,521734.78,2100.0,231164.43
3,130333.360000,108260.810000,21932.621630,16160.533926,0,2024-06-01,0.05,"[{'market_value': 522604.34, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,260526.791630,493282.451630,2576.48,522604.34,2100.0,232755.66
4,130876.420000,108621.680000,7818.219257,16214.402372,0,2024-07-01,0.05,"[{'market_value': 523475.35, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,247316.319257,481669.269257,2576.48,523475.35,2100.0,234352.95
5,131421.740000,102633.518877,0.000000,16268.450380,0,2024-08-01,0.05,"[{'market_value': 524347.81, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,234055.258877,470011.588877,2576.48,524347.81,2100.0,235956.33
6,131969.330000,88752.951452,0.000000,16322.678548,0,2024-09-01,0.05,"[{'market_value': 525221.72, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,220722.281452,458288.101452,2576.48,525221.72,2100.0,237565.82
7,132519.200000,74771.702523,0.000000,16377.087477,0,2024-10-01,0.05,"[{'market_value': 526097.09, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,207290.902523,446472.372523,2576.48,526097.09,2100.0,239181.47
8,133071.360000,60689.262232,0.000000,16431.677768,0,2024-11-01,0.05,"[{'market_value': 526973.92, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,193760.622232,434563.922232,2576.48,526973.92,2100.0,240803.30
9,133625.820000,46505.109973,0.000000,16486.450027,0,2024-12-01,0.05,"[{'market_value': 527852.21, 'monthly_income':...",0.04,0.04,0.02,True,50000,80-20,180130.929973,422562.279973,2576.48,527852.21,2100.0,242431.35


In [5]:
print(f"Without any income and burning only your investments, you can survive for {number_of_records} months")



Without any income and burning only your investments, you can survive for 38 months


In [6]:
# I want to use plotly to create a nice stacked bar chart of bonds and stock investments
# with a line chart showing monthly expenses

fig = px.bar(df, x='date', y=['properties_net_cash_value', 'stock_investments', 'bonds_investments', 'cash', ], title='Wealth over time')
fig.add_scatter(x=df['date'], y=df['monthly_expenses'], mode='lines', name='Monthly expenses')
fig.add_scatter(x=df['date'], y=df['wealth_inc_properties'], mode='lines', name='Wealth with net properties')

fig.show()
